# Case

Naive sentiment analysis

This is a simple project using Naive Bayes Classifier and Scikit-learn to create a Google Play store reviews classifier (Sentiment Analysis) in Python. You will categorize user reviews as good or bad. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. In this dataset, we use the 23 most popular mobile apps, and only two columns.

Step 1:

We have three columns: package name, review and polarity (0 = bad, 1 = good) Preprocess the data by eliminating the package name column and putting all reviews in lower case.

Step 2:

Separate target from feature, and split your data.

Step 3:

Vectorize your features and use Naive Bayes to classify the reviews as good or bad. We will not focus on hypertuning our model this time. This was an introduction project to sentiment analysis using Naive Bayes.

Step 4:

Use the app.py to create your pipeline.

Save your naive bayes classification model in the 'models' folder.

Insall de libraries

In [1]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install nltk
!pip install matplotlib
%pip install pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 97.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 93.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 47.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 42.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 96.0 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=76ef1b7bfab720be08984ac5b539f117f400bb3d705573a37c822bcc04023972
  Stored in directory: /home/gitpod/.cache/pip/wheels/22/0b/40/f

Imports

In [2]:
import pandas as pd
import numpy as np
import nltk #text processing
import re
import unicodedata
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB



In [3]:
#load the words to use as filter
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Load Data of Reviews

In [4]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

Explore Dataset

In [5]:
df.head()


,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [6]:
df['polarity'].value_counts()

0    584
1    307
Name: polarity, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


Package name, review and polarity (0 = bad, 1 = good)

In [8]:
df.sample(10)

,package_name,review,polarity
73,com.twitter.android,love it you are able to see celebrity tweets ...,1
207,com.supercell.clashofclans,want to get all 5 stars ????? reduce the amou...,0
788,org.mozilla.firefox,shite. crashes constantly! test 1st does noth...,0
627,com.uc.browser.en,outstanding this app is not only fabulous on ...,1
27,com.facebook.katana,doesn't work 90% of the time. doesn't update ...,0
113,com.linkedin.android,groups??? ** edit: i changed my rating from o...,1
477,com.Slack,it sucks! i am always shown offline and unab...,0
873,com.rovio.angrybirds,such an awesome game love it a really fun gam...,1
341,com.viber.voip,voice messages problem most times recorded me...,0
119,com.king.candycrushsaga,why do u do this to me ok so i can't unlock t...,0



Text Processing - Clean

Step 1 - change all to lower case


In [9]:
#Preprocess the data by eliminating the package name column and putting all reviews in lower case.

df.drop(['package_name'],axis=1, inplace=True)
#chante all to lower case
df['review'] = df['review'].str.lower()




Step 2 - Remove all words that not have meaning to the search - stop words


In [10]:
#see, the word in the reviews, create a list of word and frecuency
df['review'].str.split(expand = True).stack().value_counts()[:60] #(show top 60)

the       1293
to        1159
i         1084
and        853
it         778
a          619
is         555
my         442
this       421
for        410
of         390
but        348
in         340
on         325
not        318
app        317
you        315
that       260
have       252
with       220
so         203
be         194
no         186
when       182
if         180
or         169
all        163
can        158
as         148
me         145
are        139
it's       138
can't      137
update     136
new        136
like       135
fix        133
please     131
good       128
game       121
one        119
use        118
just       117
get        112
its        111
very       109
more       107
was        104
there      104
don't      103
now        102
will       102
even       101
from       101
at         100
up          99
great       99
time        93
your        90
do          89
dtype: int64

In [11]:
#remove the stopword
stop = stopwords.words('english')

def remove_stopwords(text):
  if text is not None:
    #list of the word in the text
    words = text.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop:
        words_filtered.append(word)
    result = " ".join(words_filtered) #join the word in a text with space separation
  else:
      result = None
  return result




Eliminamos los caracteres desconocidos


In [12]:
# elimina espacio libre al principio y al final
df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.lower()


In [13]:
df['review']


0      privacy at least put some option appear offlin...
1      messenger issues ever since the last update, i...
2      profile any time my wife or anybody has more t...
3      the new features suck for those of us who don'...
4      forced reload on uploading pic on replying com...
                             ...                        
886    loved it i loooooooooooooovvved it because it ...
887    all time legendary game the birthday party lev...
888    ads are way to heavy listen to the bad reviews...
889    fun works perfectly well. ads aren't as annoyi...
890    they're everywhere i see angry birds everywher...
Name: review, Length: 891, dtype: object

In [14]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [15]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)

In [16]:
df.sample(10)

,review,polarity
486,when you'll add gif support i'll give you 5 st...,1
459,wrong details of last seen after upgrading to ...,0
829,j khojyo thyehi paine ek matra apps,1
729,light fast and comfortable apps when i compare...,1
841,all nepali must use,1
582,evernote...forever evernote helps this busy mo...,1
136,the scoring system on facebook gains more poin...,0
229,sucks the new update especially the multiplaye...,0
262,the best i agree totally with the last poster....,1
0,privacy at least put some option appear offlin...,0


In [17]:
df.iloc[675,]

review      loved it rzrl app rrlrrl rrsrzrlrrl r rrsrzrlr...
polarity                                                    1
Name: 675, dtype: object

In [18]:
df_interim = df.copy()

In [19]:
df_interim['review']=df_interim['review'].str.lower()

In [20]:
df_interim['review'] = df_interim['review'].apply(remove_stopwords)


In [21]:
df_interim['review'].str.split(expand=True).stack().value_counts()[:60] 

app         351
update      150
game        140
please      138
like        137
can't       137
new         137
fix         135
good        134
use         123
one         120
great       115
get         112
even        102
time         98
browser      85
really       84
back         81
still        80
would        80
love         77
make         74
5            71
work         71
also         69
see          69
using        68
phone        68
i'm          67
open         66
it.          65
want         64
google       64
much         62
every        61
version      60
option       59
u            58
way          57
app.         56
go           56
android      56
used         55
best         54
download     53
better       53
play         52
.            51
give         50
send         49
people       48
able         48
message      47
since        47
many         46
works        45
apps         45
ads          45
keep         45
old          44
dtype: int64

In [22]:
def word_only_letters(word):
    for c in word:
        cat = unicodedata.category(c)
        if cat not in ('Ll','Lu'):  #only letter upper y lower
            return False
    return True

# clean only letter
def text_only_letters(text):
    if text is not None:
        #list of the word in the text
        words = text.strip().split()
        words_filtered = []
        for word in words:
            if word_only_letters(word):
                words_filtered.append(word)
            result = " ".join(words_filtered) #join the word in a text with space separation
    else:
        result = None
    return result

In [23]:
df_interim['review'] = df_interim['review'].apply(text_only_letters)

In [24]:
#remove multi letter looove iiiitttt, or repeat secuence
def replace_multiple_letters(message):
  if message is not None:
    result = re.sub(r'(.+?)\1+', r'\1', message)
    #result = re.sub(r"([a-zA-Z])＼1{2,}", r"＼1", message)
  else:
    result = None
  return result


In [25]:
df_interim['review'] = df_interim['review'].apply(replace_multiple_letters)

In [26]:
df_interim['review']

0      privacy least put option apear mean people lik...
1      mesenger isues ever since last update initial ...
2      profile time wife anybody one post view would ...
3      new features suck us working back buton guys m...
4      forced reload uploading pic replying coment la...
                             ...                        
886    loved incredible awesome go power make new cla...
887    time legendary game birthday party levels shor...
888    ads way heavy listen bad ads every round wheth...
889    fun works perfectly ads anoying think especial...
890    everywhere se angry birds everywhere stop play...
Name: review, Length: 891, dtype: object

In [27]:
#copy to df
df = df_interim.copy()




Setp 3 - Model


In [28]:
# Separate target and predictor
X = df['review']
y = df['polarity']


In [29]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(X)
vectorizer.get_feature_names_out()


array(['abesta', 'abilities', 'ability', ..., 'zespole', 'zom', 'zoming'],
      dtype=object)

In [30]:
X = X.toarray()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=25)

In [32]:
#Vectorizer change matriz with 0-1
#vec = CountVectorizer(stop_words = "english")
#X_train = vec.fit_transform(X_train).toarray()
#X_test = vec.transform(X_test).toarray()

In [33]:
print(X_train)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [34]:
print(X.shape)
print(X_test.shape)
print(X_train.shape)

(891, 3289)
(223, 3289)
(668, 3289)


In [35]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [36]:
y_train.value_counts()

0    438
1    230
Name: polarity, dtype: int64

In [37]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [38]:
target_names = ['bad', 'good']
print(classification_report(y_train, y_train_pred, target_names=target_names))

              precision    recall  f1-score   support

         bad       1.00      0.98      0.99       438
        good       0.96      1.00      0.98       230

    accuracy                           0.99       668
   macro avg       0.98      0.99      0.99       668
weighted avg       0.99      0.99      0.99       668



In [39]:
print(classification_report(y_test, y_test_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.80      0.83      0.81       146
        good       0.65      0.61      0.63        77

    accuracy                           0.75       223
   macro avg       0.73      0.72      0.72       223
weighted avg       0.75      0.75      0.75       223



No resultó muy bueno - overfit

# Naive Bayes

Naive Bayes is a probabilistic algorithm for classification, based on Bayes Theorem. This algorithm really makes you understand a big deal about classification and it is probably one of the most useful algorithms for simple Machine Learning use cases. It is simple enough to be completed by hand.

In [40]:
clf_multinomial = MultinomialNB()
clf_multinomial.fit(X_train, y_train)

MultinomialNB()

In [41]:
MultinomialNB()


MultinomialNB()

In [42]:
y_train_pred = clf_multinomial.predict(X_train)
y_test_pred = clf_multinomial.predict(X_test)


In [43]:
print(classification_report(y_train, y_train_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.95      0.97      0.96       438
        good       0.95      0.90      0.92       230

    accuracy                           0.95       668
   macro avg       0.95      0.94      0.94       668
weighted avg       0.95      0.95      0.95       668



In [44]:
print(classification_report(y_test, y_test_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.86      0.90      0.88       146
        good       0.79      0.71      0.75        77

    accuracy                           0.83       223
   macro avg       0.82      0.81      0.81       223
weighted avg       0.83      0.83      0.83       223



In [45]:
Z = vectorizer.transform(['I like this app'])

In [46]:
print(np.array(Z))
print(Z.shape)
print(Z)

  (0, 142)	1
  (0, 1602)	1
(1, 3289)
  (0, 142)	1
  (0, 1602)	1


In [47]:
# Modelo GaussianNB
print(clf.predict(Z.toarray()))



[1]


In [48]:
# Modelo Multinomial
print(clf_multinomial.predict(Z.toarray()))


[1]


In [49]:
pd.set_option('display.max_colwidth', None)
df.tail(10)

,review,polarity
881,game ruined felt like playing awhile imediately second awful mobile game ad poped second level loking store realized pay remove yep,0
882,ads way many ads even enjoy game amount ads every level,0
883,great game many ads almost worth,1
884,fun hard angry birds realy fun video game people least year olds people lot years older one really chalenge teach children control even though ba litle,1
885,many ads far adverts game know fre ned ads make profit neds,1
886,loved incredible awesome go power make new clash clans thing but beter,1
887,time legendary game birthday party levels short fuse levels pigs crash onto diferent chemicals players cringe much ads close ur conection play ads trouble,1
888,ads way heavy listen bad ads every round whether pas fail sometimes ads next round starts spend time ads game develop web aps honestly many people rely ads make apreciate however developers went far beyond frankly disrespectful,0
889,fun works perfectly ads anoying think especialy fre,1
890,everywhere se angry birds everywhere stop playing get head devs perfect,1


In [50]:
Z2 = vectorizer.transform(['I do not like this app'])

# Modelo GaussianNB
print(f"Gauss: {clf.predict(Z2.toarray())}")

# Modelo Multinomial
print(f"Multi: {clf_multinomial.predict(Z2.toarray())}")



Gauss: [1]
Multi: [1]


Guardar Modelo

In [51]:
pickle.dump(clf_multinomial, open('../models/multinomial.pkl', 'wb'))

Traer el Modelo

In [52]:
load_model = pickle.load(open('../models/multinomial.pkl', 'rb'))


In [53]:
Z2 = vectorizer.transform(['I do not like this app'])
# Modelo Multinomial loaded
print(f"Multi: {load_model.predict(Z2.toarray())}")



Multi: [1]


In [54]:
!pip install torch
!pip install tensorflow
!pip install -q transformers

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 130.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 128.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 120.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 46.0 MB/s eta 0:00:00
     ━━━━

/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-09 01:00:47.310059: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-09 01:00:47.310094: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [55]:
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [56]:
nb.predict(vec.transform(['i don\'t like','eat much ram','eat','like']))


NameError: name 'nb' is not defined

In [ ]:
nb.predict(vec.transform(['love','great solid love important feature','ram','like fast']))

array([1, 1, 0, 1])

In [ ]:
df[df['polarity']==1]['review'].sample(10)

679                              hamro keyboard god ap doubt however ading features definitely make best folowing things option caps lock key enabled made available leters available along also les options go numbers option one signs mising easy togle option phone keyboard ap keyboard made post
349                                                 viber voice mesages chating viber best ap caling services even mesenger performs ap optimised newest version new option introduced alows se friends sen mesage group wel working makes litle bit mad finaly function introduced viber stil working
701                                                                                                                                      stil best mini like new lok news front fast eficient though overal best around browsers id go maxthon yandex opera mini gold though uc mini coming along neds
182                                                                                                                

In [ ]:
df[df['polarity']==0]['review'].sample(10)

65              neds actual update twiter slows twets continue even takes long time load twet use moblie site version actualy use
166                                                      like review take advice cant save data though always find chest uninstal
308                          download pics used able download pics conversation save option long pres gives every option download
338    option separate viber se viber contacts separately shows together last update god cal drops video cal hear person properly
427                                     se change friend change nicknames mine setings change color anything fix please faster gr
36                             update uninstals everytime try update uninstals ap want know using data time update driving please
392                    major anoyance new update show online define online gogle using timestamps please ad dark mode feature wel
699                                                     cant download big things cant down

Save the model 


In [ ]:
#filename = '../modelo_final.sav' #use absolute path
#pickle.dump(nb, open('../models/modelo_final.sav', 'wb'))
